# Data Cleanup

Data cleanup methods refer to book "Data Wrangling with Python" and "Python for Data Analysis".

In [156]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare the data

This data set related to child labor. The data is stored in mn.cvs with column names in acronyms. We can find these acronyms in the mn_headers.csv.

In [157]:
data = pd.read_csv('/Users/newuser/Desktop/Research(Tian_Ye)/Learning_notes/data-wrangling/data/unicef/mn.csv', index_col = 0, engine = 'python')
data.head()

,HH1,HH2,LN,MWM1,MWM2,MWM4,MWM5,MWM6D,MWM6M,MWM6Y,...,MCSURV,MCDEAD,mwelevel,mnweight,wscore,windex5,wscoreu,windex5u,wscorer,windex5r
1,1,17,1,1,17,1,14,7,4,2014,...,0.0,0.0,Higher,0.403797,1.603670,5,1.272552,5.0,NaN,NaN
2,1,20,1,1,20,1,14,7,4,2014,...,0.0,0.0,Higher,0.403797,1.543277,5,1.089026,5.0,NaN,NaN
3,2,1,1,2,1,1,9,8,4,2014,...,3.0,0.0,Primary,1.031926,0.878635,4,-0.930721,1.0,NaN,NaN
4,2,1,5,2,1,5,9,12,4,2014,...,NaN,NaN,NaN,0.000000,0.000000,0,0.000000,0.0,0.0,0.0
5,2,1,8,2,1,8,9,8,4,2014,...,0.0,0.0,Secondary,1.031926,0.878635,4,-0.930721,1.0,NaN,NaN


**Remark**:
1. The default `engine` is `'C'` which is faster but perform worse than `'python'` when there are multiple data types in the data set.
2. Set `index_col = 0` to let the first column as the index.

In [158]:
data_header = pd.read_csv('/Users/newuser/Desktop/Research(Tian_Ye)/Learning_notes/data-wrangling/data/unicef/mn_headers.csv')
data_header.head()

,Name,Label,Question
0,HH1,Cluster number,NaN
1,HH2,Household number,NaN
2,LN,Line number,NaN
3,MWM1,Cluster number,NaN
4,MWM2,Household number,NaN


## Create a new data with informative columns names

The data has 159 columns while data_header has has 210 rows. Check how many column names in data can find a description in data_header.

In [160]:
sum(data.columns.isin(data_header['Name']))

150

In [164]:
d = data.columns[data.columns.isin(data_header['Name'])]
f = data_header['Label'][data_header['Name'].isin(d)]

In [173]:
np.where(~f.isin(data_new.columns))

(array([], dtype=int64),)

In [176]:
sum(data_header['Name'].duplicated())

0

Create a data to contain the columns whose name can be found in the data_header.

In [210]:
data_new = DataFrame()
n = data.shape[0]
for i in range(data.shape[1]):
    col_new = data_header['Label'][data_header['Name'] == data.columns[i]]
    #Note that col_new has dtype: object and hence cannot be used as column name.   
    if len(col_new) > 0:
        col_new = col_new.iloc[0]     #Apply .iloc to convert the dtype: object to normal string.
                                        #We choose index 0 because there is only one element in the list.
        if len(data_new) != 0:
            data_new = pd.concat([data_new, data.iloc[:, i]], axis = 1) #concatenate the data by rows
        else:
            data_new[col_new] = data.iloc[:, i]
data_new.head()

,Cluster number,HH2,LN,MWM1,MWM2,MWM4,MWM5,MWM6D,MWM6M,MWM6Y,...,MWAGE,MWDOM,MWAGEM,MWDOBLC,MMSTATUS,MCEB,MCSURV,MCDEAD,wscore,windex5
1,1,17,1,1,17,1,14,7,4,2014,...,25-29,1365.0,29.0,NaN,Currently married/in union,0.0,0.0,0.0,1.603670,5
2,1,20,1,1,20,1,14,7,4,2014,...,35-39,1370.0,37.0,NaN,Currently married/in union,0.0,0.0,0.0,1.543277,5
3,2,1,1,2,1,1,9,8,4,2014,...,40-44,1100.0,18.0,NaN,Currently married/in union,3.0,3.0,0.0,0.878635,4
4,2,1,5,2,1,5,9,12,4,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0
5,2,1,8,2,1,8,9,8,4,2014,...,20-24,NaN,NaN,NaN,Never married/in union,0.0,0.0,0.0,0.878635,4


**Remark**: 
1. When we use boolean index to find some specific targets from one column of strings, we always get results with dtype: object. But I cannot find a way to get value from object type (get the string from it). So I use `.iloc()` to convert the data type but it is not effective. 
2. We can use [`data.where()`](http://pandas.pydata.org/pandas-docs/stable/indexing.html) to get the index first. It outputs one tuple data type. However, we need to use int() to convert np.array type to int. The disadvantage of this method is that it failed to deal with all False situation.

In [115]:
i=1
int(np.where(data_header['Name'] == data.columns[i])[0])

1

## Formatting Data

Print out the results with readable format.

In [118]:
print('Question: {}\nAnswer: {}'.format(data_new.columns[0], data_new.iloc[0, 0]))

Question: Cluster number
Answer: 1


Other options.

In [121]:
example_dict = {'f': 2.123123123131,
               'i': 3433233423423,
               'p': .324,}
s_to_p = 'float: {f:.4f}\n'
s_to_p += 'integer: {i:,}\n'
s_to_p += 'percentatage: {p:.2%}'
print (s_to_p.format(**example_dict)) #use ** tounpack the dictionary.

float: 2.1231
integer: 3,433,233,423,423
percentatage: 32.40%


## Date Operations

Firstly, let's look at our data holding interiew start and end times from `data_new`. Print some of our entries to make sure we know what data entries we need to use:

In [126]:
data_new.iloc[0, 6:15]
for x in enumerate(data_new.columns[:15]):
    print(x)           

(0, 'Cluster number')
(1, 'Household number')
(2, 'Line number')
(3, "Man's line number")
(4, 'Interviewer number')
(5, 'Day of interview')
(6, 'Month of interview')
(7, 'Year of interview')
(8, "Result of man's interview")
(9, 'Field editor')
(10, 'Data entry clerk')
(11, 'Start of interview - Hour')
(12, 'Start of interview - Minutes')
(13, 'End of interview - Hour')
(14, 'End of interview - Minutes')


**Remark**: `enumerate` can let's see the positions of entries.

We now have all the data we need to figure out exctly when the interview started and ended. We could use data like this to determine things whether interviews in th evening or morning were more likely to be completed, and whether the length of the interview affected the number of rsponses. We can also determine which was the first interview and the last interview and calculated average duration.

Now let's get the start time.

In [131]:
from datetime import datetime

start_string = '{}/{}/{} {}:{}'.format(data_new.iloc[0, 6], 
                                      data_new.iloc[0, 5], data_new.iloc[0, 7], int(data_new.iloc[0, 11]), 
                                      int(data_new.iloc[0, 12]))
start_time = datetime.strptime(start_string, '%m/%d/%Y %H:%M')
start_time

datetime.datetime(2014, 4, 7, 17, 59)

**Remark**: When we use datetime.strptime, we should make sure all inputs are integers.

Since each element of the time data is a sparate item in our dataset, we could also natively create Python datetime objects without using `strptime`.

In [135]:
end_time = datetime(data_new.iloc[0, 7], data_new.iloc[0, 6], data_new.iloc[0, 5], 
                    int(data_new.iloc[0, 13]), int(data_new.iloc[0, 14]))
end_time

datetime.datetime(2014, 4, 7, 18, 7)

**Remark**: the order of inputs for `datetime` should be year, month, day, hour and minute.

Now we can get some information from date data.

In [138]:
duration = end_time - start_time
print(duration)

0:08:00


In [140]:
print(duration.days)

0


In [141]:
print(duration.total_seconds()) #It is equivalent to directly type `duration`.

480.0


In [142]:
minutes = duration.total_seconds()/60.0
minutes

8.0

Present the date in a human-readable way:

In [143]:
print(end_time.strftime('%m/%d/%Y %H:%M:%S'))

04/07/2014 18:07:00


In [144]:
print(start_time.ctime())   #C's ctime standard

Mon Apr  7 17:59:00 2014


In [150]:
print(start_time.strftime('%Y-%m-%dT%H:%M:%S'))  #PHP format

2014-04-07T17:59:00


## Finding bad data

### Finding NA in each column

Firstly, let's check how many NA or NONE value data in each column.

In [212]:
num_na = np.sum(data_new.isnull())

In [217]:
num_na.describe()

count     150.000000
mean     3868.420000
std      3233.806999
min         0.000000
25%      1094.000000
50%      2931.000000
75%      6782.000000
max      9008.000000
dtype: float64

Check data types.

In [219]:
data_new.dtypes

Cluster number      int64
HH2                 int64
LN                  int64
MWM1                int64
MWM2                int64
MWM4                int64
MWM5                int64
MWM6D               int64
MWM6M               int64
MWM6Y               int64
MWM7               object
MWM8                int64
MWM9                int64
MWM10H            float64
MWM10M            float64
MWM11H            float64
MWM11M            float64
MWB1M             float64
MWB1Y             float64
MWB2              float64
MWB3               object
MWB4               object
MWB5              float64
MWB7               object
MMT2               object
MMT3               object
MMT4               object
MMT6               object
MMT7               object
MMT8               object
                   ...   
MTA14              object
MTA15             float64
MTA16             float64
MTA17             float64
TNLN              float64
TN4                object
TN5                object
TN6         

### Finding duplicates

For `DataFrame`, we can use `pd.duplicated()`. This can check all rows, if two rows are exactly same it outputs True.

We can delete redundant dulicates by `.drop_duplicates()`.

In [222]:
sum(data_new.duplicated())

0

### Fuzzy match

Sometimes, we want to treat "My dog & I" and "me and my dog" as same. We can apply fuzzywuzzy package to do this.

In [226]:
from fuzzywuzzy import fuzz

/Users/newuser/anaconda/lib/python3.5/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [227]:
a = DataFrame([[1,2],[3,4]])
fuzz.ratio(a, a)

100

There are many other options for fuzzywuzzy package:

1. `fuzz.token_sort_ratio`: ignore the word order. E.g. "dog vs. cat" and "cat vs. dog".
2. `process`: find best match.

In [230]:
from fuzzywuzzy import process
choices = ['Yes', 'No', 'Maybe', 'N/A']
process.extract('ya', choices, limit = 2) #Find two best matches

[('Yes', 45), ('Maybe', 45)]

In [231]:
process.extractOne('ya', choices)

('Yes', 45)

### Regular expressions match